In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import scipy.stats as st
from smart_contract import *

In [3]:
# ETH balance
owner, SP, policyholder, contract = [0.4366], [0.2], [0.21], [0]
# Smart Contract variable
totalSupply, rt, sn2, mu, SCR, MCR, Xt, eta1, eta2, qAlphaSCR, qAlphaMCR, Nt = [0], [1], [0], [0], [0], [0], [0], 0.1, 0.05, st.norm.ppf(0.995), st.norm.ppf(0.85), 0
eta=eta1 + eta2
Yt = {}


Event:
 - Contract owner invest x1 = ETH0.08 
 - SP invest x2 = ETH0.1

In [4]:
x1 = 0.08
owner.append(owner[-1] - x1)
contract.append(contract[-1] + x1)
Xt.append(Xt[-1] + x1)
y1 = x1 / rt[-1]
Yt["owner"] = [y1]
totalSupply.append(totalSupply[-1] + y1)

x2 = 0.1
SP.append(SP[-1] - x2)
contract.append(contract[-1] + x2)
Xt.append(Xt[-1] + x2)
y2 = x2 / rt[-1]
Yt["SP"] = [y2]
totalSupply.append(totalSupply[-1] + y2)


Xt[-1], Yt, totalSupply[-1]


(0.18, {'owner': [0.08], 'SP': [0.1]}, 0.18)

Event:
- Underwrite ("rain"; 0.06)
- Underwrite ("hail", 0.01)

In [5]:
PTF = []
event1 = "rain"
l1, p1 = 0.06, get_random_number(event1) 
r1 = (1 + eta1) * p1 * l1
cp1 = (1 + eta) * p1 * l1
policy1 = InsuranceContract(
    customer="policyholder",
    event_description=b"rain",
    l=l1,
    p=p1,
    eta=eta1 + eta2,
    refund=r1,
    status=0
)
PTF.append(policy1)
contract.append(contract[-1] + cp1)
Yt["policyholder"] = [0]
Yt["policyholder"] = [Yt["policyholder"][-1] + eta2 * p1 * l1 / rt[-1]]  
sn2.append(sn2[-1] + (l1 ** 2) * p1 * (1 - p1)), mu.append(mu[-1] + p1 * l1 * eta) 
SCR.append(np.sqrt(sn2[-1]) * qAlphaSCR - mu[-1]), MCR.append(np.sqrt(sn2[-1]) * qAlphaMCR - mu[-1])
policyholder.append(policyholder[-1] - cp1)
totalSupply.append(totalSupply[-1] + Yt["policyholder"][-1])
rt.append(Xt[-1] / totalSupply[-1])

print(SCR[-1], MCR[-1], contract[-1], Xt[-1], totalSupply[-1], rt[-1])

event2 = "hail"
l2, p2 = 0.01, get_random_number(event2) 
r2 = (1 + eta2) * p2 * l2
cp2 = (1 + eta) * p2 * l2
policy2 = InsuranceContract(
    customer="policyholder",
    event_description=b"hail",
    l=l2,
    p=p2,
    eta=eta1 + eta2,
    refund=r2,
    status=0
)

PTF.append(policy2)
contract.append(contract[-1] + cp2)
Yt["policyholder"].append(Yt["policyholder"][-1] + eta2 * p2 * l2 / rt[-1])
sn2.append(sn2[-1] + (l2 ** 2) * p2 * (1 - p2)), mu.append(mu[-1] + p2 * l2 * eta) 
SCR.append(np.sqrt(sn2[-1]) * qAlphaSCR - mu[-1]), MCR.append(np.sqrt(sn2[-1]) * qAlphaMCR - mu[-1])
policyholder.append(policyholder[-1] - cp2)
totalSupply.append(totalSupply[-1] + eta2 * p2 * l2 / rt[-1])
rt.append(Xt[-1] / totalSupply[-1])

print(SCR[-1], MCR[-1], contract[-1], Xt[-1], totalSupply[-1], rt[-1])

0.06321492818983102 0.021582429052397788 0.22943159999999999 0.18 0.18214919999999998 0.9882008814751864
0.06331281011317799 0.021593575612694316 0.22979385 0.18 0.18216513805499998 0.9881144214633083


Event:
 - Parameter change eta1 = 0.025 et eta2 = 0.025
 - underwrite("snow, 0.1")


In [6]:
eta1, eta2 = 0.025, 0.025
eta = eta1 + eta2

event3 = "snow"
l3, p3 = 0.1, get_random_number(event3) 
r3 = (1 + eta2) * p3 * l3
cp3 = (1 + eta) * p3 * l3
policy3 = InsuranceContract(
    customer="policyholder",
    event_description=b"snow",
    l=l3,
    p=p3,
    eta=eta1 + eta2,
    refund=r3,
    status=0
)
PTF.append(policy3)
contract.append(contract[-1] + cp3)
Yt["policyholder"].append(Yt["policyholder"][-1] + eta2 * p3 * l3 / rt[-1])
sn2.append(sn2[-1] + (l3 ** 2) * p3 * (1 - p3)), mu.append(mu[-1] + p3 * l3 * eta) 
SCR.append(np.sqrt(sn2[-1]) * qAlphaSCR - mu[-1]), MCR.append(np.sqrt(sn2[-1]) * qAlphaMCR - mu[-1])
policyholder.append(policyholder[-1] - cp3)
totalSupply.append(totalSupply[-1] + eta2 * p3 * l3 / rt[-1])
rt.append(Xt[-1] / totalSupply[-1])


print(SCR[-1], MCR[-1], Xt[-1], contract[-1], totalSupply[-1], rt[-1])


0.13709083390200916 0.04968746228867339 0.18 0.28573785 0.18351316007660698 0.9808560864237724


Event:
- Settle ('hail', 'hail1' ,0.01) => false
- owner withdraw 0.01

In [7]:
PTF[1].status = 2
Xt.append(Xt[-1] + cp2)
rt.append(Xt[-1] / totalSupply[-1])
sn2.append(sn2[-1] - (l2 ** 2) * p2 * (1 - p2)), mu.append(mu[-1] - p2 * l2 * PTF[1].eta) 
SCR.append(np.sqrt(sn2[-1]) * qAlphaSCR - mu[-1]), MCR.append(np.sqrt(sn2[-1]) * qAlphaMCR - mu[-1])

print(SCR[-1], MCR[-1], contract[-1], Xt[-1], totalSupply[-1], rt[-1])

y3 = 0.01
x3 = y3 * rt[-1]
owner.append(owner[-1] + x3)
contract.append(contract[-1] - x3)
Xt.append(Xt[-1] - x3)
Yt["owner"].append(Yt["owner"][-1] - y3)
totalSupply.append(totalSupply[-1] - y3)

print( contract[-1], Xt[-1], totalSupply[-1], SCR[-1], owner)

0.13706886540068072 0.049706860922183904 0.28573785 0.18036225 0.18351316007660698 0.9828300592977003
0.275909549407023 0.170533949407023 0.17351316007660697 0.13706886540068072 [0.4366, 0.3566, 0.366428300592977]


Event:
- settle (snow, 0.1) =>  snow2 => true
- underwrite("sun", 0.08)

In [8]:
PTF[2].status = 1
Xt.append(Xt[-1] + cp3 - l3)
rt.append(Xt[-1] / totalSupply[-1])
sn2.append(sn2[-1] - (l3 ** 2) * p3 * (1 - p3)), mu.append(mu[-1] - p3 * l3 * PTF[2].eta) 
SCR.append(np.sqrt(sn2[-1]) * qAlphaSCR - mu[-1]), MCR.append(np.sqrt(sn2[-1]) * qAlphaMCR - mu[-1])
policyholder.append(policyholder[-1] + l3)
contract.append(contract[-1] - l3)

print(SCR[-1], MCR[-1], contract[-1], Xt[-1])

event4 = "sun"
l4, p4 = 0.08, get_random_number(event4) 
r4 = (1 + eta2) * p4 * l4
cp4 = (1 + eta) * p4 * l4
policy4 = InsuranceContract(
    customer="policyholder",
    event_description=b"sun",
    l=l4,
    p=p4,
    eta=eta1 + eta2,
    refund=r4,
    status=0
)
PTF.append(policy4)
contract.append(contract[-1] + cp4)
Yt["policyholder"].append(Yt["policyholder"][-1] + eta2 * p4 * l4 / rt[-1])
sn2.append(sn2[-1] + (l4 ** 2) * p4 * (1 - p4)), mu.append(mu[-1] + p4 * l4 * eta) 
SCR.append(np.sqrt(sn2[-1]) * qAlphaSCR - mu[-1]), MCR.append(np.sqrt(sn2[-1]) * qAlphaMCR - mu[-1])
policyholder.append(policyholder[-1] - cp4)
totalSupply.append(totalSupply[-1] + eta2 * p4 * l4 / rt[-1])
rt.append(Xt[-1] / totalSupply[-1])

print(SCR[-1], MCR[-1], contract[-1], Xt[-1])


0.06321492818983103 0.02158242905239779 0.175909549407023 0.12647794940702298
0.11511973698502277 0.04146615380265091 0.211088749407023 0.12647794940702298


Event:
- Investor withdraw as much as possible
- Change parameter MCR
- settle ("sun", 0.08) => "sun1" => True

In [9]:
y4 = (Xt[-1] - SCR[-1]) / rt[-1]

x4 = y4 * rt[-1]
print(y4, x4)
SP.append(SP[-1] + x4)
contract.append(contract[-1] - x4)
Xt.append(Xt[-1] - x4)
Yt["SP"].append(Yt["SP"][-1] - y4)
totalSupply.append(totalSupply[-1] - y4)
print(contract[-1], Xt[-1], totalSupply[-1], rt[-1], SCR[-1])



qAlphaMCR = st.norm.ppf(0.985)
print(qAlphaMCR)
MCR.append(np.sqrt(sn2[-1]) * qAlphaMCR - mu[-1])
print(MCR[-1], Xt[-1])


PTF[3].status = 1
Xt.append(Xt[-1] + cp4 - l4)
rt.append(Xt[-1] / totalSupply[-1])
sn2.append(sn2[-1] - (l4 ** 2) * p4 * (1 - p4)), mu.append(mu[-1] - p4 * l4 * PTF[2].eta) 
SCR.append(np.sqrt(sn2[-1]) * qAlphaSCR - mu[-1]), MCR.append(np.sqrt(sn2[-1]) * qAlphaMCR - mu[-1])
policyholder.append(policyholder[-1] + l4)
contract.append(contract[-1] - l4)


print(SCR[-1], MCR[-1], contract[-1], Xt[-1], SP)




0.01568535032865742 0.011358212422000205
0.1997305369850228 0.11511973698502277 0.15897690034881357 0.724128705066188 0.11511973698502277
2.17009037758456
0.09570684556379032 0.11511973698502277
0.06321492818983103 0.05224184106454666 0.1197305369850228 0.07029893698502278 [0.2, 0.1, 0.11135821242200021]


Event:
- Underwrite (0.02, "wind")
- Change parameter SCR
- owner withdraw as much as he can
- settle (0.02, "wind") => "wind9" => True

In [10]:
event5 = "wind"
l5, p5 = 0.02, get_random_number(event5) 
r5 = (1 + eta2) * p5 * l5
cp5 = (1 + eta) * p5 * l5
policy5 = InsuranceContract(
    customer="policyholder",
    event_description=b"wind",
    l=l5,
    p=p5,
    eta=eta1 + eta2,
    refund=r5,
    status=0
)
PTF.append(policy5)
contract.append(contract[-1] + cp5)
Yt["policyholder"].append(Yt["policyholder"][-1] + eta2 * p5 * l5 / rt[-1])
sn2.append(sn2[-1] + (l5 ** 2) * p5 * (1 - p5)), mu.append(mu[-1] + p5 * l5 * eta) 
SCR.append(np.sqrt(sn2[-1]) * qAlphaSCR - mu[-1]), MCR.append(np.sqrt(sn2[-1]) * qAlphaMCR - mu[-1])
policyholder.append(policyholder[-1] - cp5)
totalSupply.append(totalSupply[-1] + eta2 * p5 * l5 / rt[-1])
rt.append(Xt[-1] / totalSupply[-1])

print(SCR[-1], MCR[-1], contract[-1], rt[-1], Xt[-1])

qAlphaSCR = st.norm.ppf(0.99)
print(qAlphaSCR)
SCR.append(np.sqrt(sn2[-1]) * qAlphaSCR - mu[-1])
print(SCR[-1], Xt[-1])


y5 = (Xt[-1] - SCR[-1]) / rt[-1]
x5 = y5 * rt[-1]
print(y5, x5)
owner.append(owner[-1] + x5)
contract.append(contract[-1] - x5)
Xt.append(Xt[-1] - x5)
Yt["owner"].append(Yt["owner"][-1] - y5)
totalSupply.append(totalSupply[-1] - y5)
print(contract[-1], Xt[-1], totalSupply[-1], SCR[-1], MCR[-1])


PTF[4].status = 1
Xt.append(Xt[-1] + cp5 - l5)
rt.append(Xt[-1] / totalSupply[-1])
sn2.append(sn2[-1] - (l5 ** 2) * p5 * (1 - p5)), mu.append(mu[-1] - p5 * l5 * PTF[2].eta) 
SCR.append(np.sqrt(sn2[-1]) * qAlphaSCR - mu[-1]), MCR.append(np.sqrt(sn2[-1]) * qAlphaMCR - mu[-1])
policyholder.append(policyholder[-1] + l5)
contract.append(contract[-1] - l5)

print(SCR[-1], MCR[-1], contract[-1], rt[-1], Xt[-1])



0.06733622089998448 0.055637797027450635 0.1298840369850228 0.4406804720231199 0.07029893698502278
2.3263478740408408
0.06014307446037603 0.07029893698502278
0.023045864678373888 0.010155862524646751
0.11972817446037604 0.06014307446037603 0.13647773903904842 0.06014307446037603 0.055637797027450635
0.056467778022698596 0.05224184106454666 0.09972817446037603 0.36853317482044007 0.050296574460376026


Event:
- Smart contract liquidation
    - Refund to policyholder   
    - distribution of the leftovers to token holder

In [11]:
Xt.append(Xt[-1] + cp5)
policyholder.append(policyholder[-1] + r5)
contract.append(contract[-1] - r5)
Xt.append(Xt[-1] - r5)
rt.append(Xt[-1] / totalSupply[-1])

keys_Yt = list(Yt.keys())
token_holdings = np.array([Yt[key_Yt][-1] for key_Yt in keys_Yt])
xs = token_holdings * rt[-1]
contract.append(contract[-1] - sum(xs)), Xt.append(Xt[-1] + sum(xs))
owner.append(owner[-1] + xs[0]), SP.append(SP[-1] + xs[1]), policyholder.append(policyholder[-1] + xs[2])

owner, SP, policyholder



([0.4366, 0.3566, 0.366428300592977, 0.3765841631176237, 0.39397149194025166],
 [0.2, 0.1, 0.11135821242200021, 0.14258030880091277],
 [0.21,
  0.1605684,
  0.16020615,
  0.10426214999999998,
  0.20426214999999998,
  0.16908294999999998,
  0.24908294999999997,
  0.23892944999999996,
  0.25892944999999995,
  0.26884119999999995,
  0.27077009925883544])

In [21]:
pd.DataFrame({
    "Contract #" : [1,2,3,4,5],
    "Event" : ["rain", "hail", "snow", "sun", "wind"],
    "Probability of the event" : [get_random_number("rain"), 
                  get_random_number("hail"), 
                  get_random_number("snow"), 
                  get_random_number("sun"), 
                  get_random_number("wind")],
    "Observed Event" : ["rain1", "hail1", "snow2", "sun1", "wind9"],
    "Probability of the observed" : [get_random_number("rain1"), 
                  get_random_number("hail1"), 
                  get_random_number("snow2"), 
                  get_random_number("sun1"), 
                  get_random_number("wind9")]}
                  )    

,Contract #,Event,Probability of the event,Observed Event,Probability of the observed
0,1,rain,0.7164,rain1,0.4519
1,2,hail,0.0315,hail1,0.4496
2,3,snow,0.5328,snow2,0.2058
3,4,sun,0.4188,sun1,0.0128
4,5,wind,0.4835,wind9,0.1940
